In [1]:
pip install ucimlrepo

In [2]:
from ucimlrepo import list_available_datasets
print(list_available_datasets())

-------------------------------------
The following datasets are available:
-------------------------------------
Dataset Name                                                                            ID    
------------                                                                            --    
Abalone                                                                                 1     
Adult                                                                                   2     
Annealing                                                                               3     
Audiology (Standardized)                                                                8     
Auto MPG                                                                                9     
Automobile                                                                              10    
Balance Scale                                                                           12    
Balloons                       

In [ ]:
import pandas as pd
import io

num_features = 856

column_names = ['class'] + [f'feature_{i}' for i in range(1, num_features + 1)]

df = pd.read_csv(io.BytesIO(uploaded['CNAE-9.data']), header=None, names=column_names)

y = df['class']
X = df.iloc[:, 1:]


NameError: name 'uploaded' is not defined

In [ ]:
import pandas as pd

file_path = "winequality-white.csv"

df = pd.read_csv(file_path, header=None, delim_whitespace=True)
df = pd.read_csv(file_path, header=None, sep=";")

X = df.iloc[1:, :-1]
y = df.iloc[1:, -1]


Prvých 5 riadkov atribútov (features):
    0     1     2     3      4   5    6       7     8     9     10
1    7  0.27  0.36  20.7  0.045  45  170   1.001     3  0.45   8.8
2  6.3   0.3  0.34   1.6  0.049  14  132   0.994   3.3  0.49   9.5
3  8.1  0.28   0.4   6.9   0.05  30   97  0.9951  3.26  0.44  10.1
4  7.2  0.23  0.32   8.5  0.058  47  186  0.9956  3.19   0.4   9.9
5  7.2  0.23  0.32   8.5  0.058  47  186  0.9956  3.19   0.4   9.9

Prvých 5 hodnot tried (targets):
1    6
2    6
3    6
4    6
5    6
Name: 11, dtype: object


<ipython-input-47-227dd00047dd>:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(file_path, header=None, delim_whitespace=True)


In [ ]:
from ucimlrepo import fetch_ucirepo
import pandas as pd

iris = fetch_ucirepo(id=53)

X = iris.data.features
y = iris.data.targets

print("metadáta sú: ", iris.metadata)

print("variables sú: ",iris.variables)


metadáta sú:  {'uci_id': 53, 'name': 'Iris', 'repository_url': 'https://archive.ics.uci.edu/dataset/53/iris', 'data_url': 'https://archive.ics.uci.edu/static/public/53/data.csv', 'abstract': 'A small classic dataset from Fisher, 1936. One of the earliest known datasets used for evaluating classification methods.\n', 'area': 'Biology', 'tasks': ['Classification'], 'characteristics': ['Tabular'], 'num_instances': 150, 'num_features': 4, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1936, 'last_updated': 'Tue Sep 12 2023', 'dataset_doi': '10.24432/C56C76', 'creators': ['R. A. Fisher'], 'intro_paper': {'ID': 191, 'type': 'NATIVE', 'title': 'The Iris data set: In search of the source of virginica', 'authors': 'A. Unwin, K. Kleinman', 'venue': 'Significance, 2021', 'year': 2021, 'journal': 'Significance, 2021', 'DOI': '1740-9713.01589', 'URL': 'https://www.seman

In [ ]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, f1_score
from sklearn.metrics import accuracy_score, make_scorer, f1_score, confusion_matrix, classification_report
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y.values.ravel())

models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Naive Bayes": GaussianNB(),
    "KNN": KNeighborsClassifier(n_neighbors=10, metric='euclidean'),
    "Neural Network": MLPClassifier(hidden_layer_sizes=(64, 32, 16), max_iter=1000,random_state=42)
}

f1 = make_scorer(f1_score, average='weighted')

probability_matrices = {}

numMAtInc=1

accuracy_vectors = {}

for name, model in models.items():
    if name == "Neural Network":
        scores = cross_val_score(model, X_scaled, y_encoded, cv=10, scoring=f1)
        y_pred = cross_val_predict(model, X_scaled, y_encoded, cv=10)
        y_proba = cross_val_predict(model, X_scaled, y_encoded, cv=10, method='predict_proba')
    else:
        scores = cross_val_score(model, X_scaled, y_encoded, cv=10, scoring=f1)
        y_pred = cross_val_predict(model, X_scaled, y_encoded, cv=10)
        y_proba = cross_val_predict(model, X_scaled, y_encoded, cv=10, method='predict_proba')

    print(f"{name} - Priemerná F1 miera: {scores.mean():.4f}, Štandardná odchýlka: {scores.std():.4f}")
    acc = accuracy_score(y_encoded, y_pred)
    accuracy_vectors[name] = acc
    print(f"{name} - Accuracy: {acc:.4f}")
    print(f"{name} - Matica klasifikácie:")
    print(confusion_matrix(y_encoded, y_pred))
    print(classification_report(y_encoded, y_pred))

    proba_df = pd.DataFrame(y_proba, columns=label_encoder.inverse_transform(np.arange(y_proba.shape[1])))
    proba_df.index = [f"ID {i+1}" for i in range(len(proba_df))]

    proba_df = proba_df.round(5)

    print(proba_df)

    proba_df.to_excel(f"{name}_probabilities.xlsx", index=True)
    print(f"Tabuľka s pravdepodobnosťami pre model {name} bola exportovaná do Excel súboru {name}_probabilities.xlsx")
    print("\n")

    probability_matrices[f"P_{numMAtInc}"] = proba_df.values

    numMAtInc=numMAtInc+1

print(accuracy_vectors)


Decision Tree - Priemerná F1 miera: 0.9530, Štandardná odchýlka: 0.0428
Decision Tree - Accuracy: 0.9533
Decision Tree - Matica klasifikácie:
[[50  0  0]
 [ 0 47  3]
 [ 0  4 46]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       0.92      0.94      0.93        50
           2       0.94      0.92      0.93        50

    accuracy                           0.95       150
   macro avg       0.95      0.95      0.95       150
weighted avg       0.95      0.95      0.95       150

        Iris-setosa  Iris-versicolor  Iris-virginica
ID 1            1.0              0.0             0.0
ID 2            1.0              0.0             0.0
ID 3            1.0              0.0             0.0
ID 4            1.0              0.0             0.0
ID 5            1.0              0.0             0.0
...             ...              ...             ...
ID 146          0.0              0.0             1.0
ID 147          

In [ ]:
print("Matica P_1:")
print(probability_matrices["P_4"])

In [ ]:
matrix_M = np.array([[[probability_matrices[f'P_{k+1}'][i, j] for k in range(len(probability_matrices))] for j in range(probability_matrices[f'P_{1}'].shape[1])] for i in range(probability_matrices[f'P_{1}'].shape[0])])

print("Nová matica:")
print(matrix_M)


NameError: name 'np' is not defined

In [ ]:
import numpy as np
from scipy.optimize import fsolve

def sugeno_lambda(singleton_measures, initial_guess=-0.9):

    def equation(lmbd):
        product = np.prod([1 + lmbd * m for m in singleton_measures])
        return product - (1 + lmbd)

    lmbd_solution = fsolve(equation, initial_guess)[0]
    return lmbd_solution


Vyriešené λ = -0.9999969991506711


In [ ]:
import numpy as np

def sugeno_measure(subset, singleton_measures):

    lmbd=sugeno_lambda(singleton_measures,-0.9)
    if np.isclose(lmbd, 0):
        return sum(singleton_measures[i] for i in subset)
    else:
        prod_val = np.prod([1 + lmbd * singleton_measures[i] for i in subset])
        return (prod_val - 1) / lmbd


Vyriešené λ = -0.9999969991506711
Hodnota fuzzy miery pre podmnožinu [0, 3] je: 0.9975


In [ ]:
def choquet_integral(f, singleton_measures):

    f = np.array(f)
    n = len(f)

    sorted_indices = np.argsort(f)
    f_sorted = f[sorted_indices]

    integral = 0.0
    f_prev = 0.0
    for i in range(n):
        diff = f_sorted[i] - f_prev
        subset = sorted_indices[i:].tolist()
        m_A = sugeno_measure(subset, singleton_measures)
        integral += diff * m_A
        f_prev = f_sorted[i]
    return integral


Hodnota Choquet integrálu = 0.893686150017065


In [ ]:
def delta_abs(x, y):
        return abs(x - y)

def delta_square(x, y):
        return (x - y)**2

def delta_sqrt(x, y):
        return abs(x**(1/2) - y**(1/2))

def delta_sqrt_square(x, y):
        return (x**(1/2) - y**(1/2))**2

def delta_sqrt_abs(x, y):
        return (abs(x - y))**(1/2)

def delta_sqrt_sqrt(x, y):
        return (x**(1/2) - y**(1/2))**(1/2)

def delta_sqrt4(x, y):
        return abs(x**(1/4) - y**(1/4))

In [ ]:
import numpy as np

def d_choquet_integral(f, singleton_measures, delta_function):

    f = np.array(f)
    n = len(f)

    sorted_indices = np.argsort(f)
    f_sorted = f[sorted_indices]

    total_integral = 0.0
    previous_value = 0.0
    for i in range(n):
        delta_val = delta_function(f_sorted[i], previous_value)
        subset = sorted_indices[i:].tolist()
        m_value = sugeno_measure(subset, singleton_measures)
        total_integral += delta_val * m_value

        previous_value = f_sorted[i]

    return total_integral


d-Choquet integrál (s využitím absolútneho rozdielu ako δ) = 0.893686150017065


In [ ]:
def DR_function(array):
    return sorted(set(array))


[1, 2, 5, 7]


In [ ]:
def pseudo_overlap_Z(x, y):

    diff = np.maximum(x - y, 0)
    return np.sqrt(diff * x) if diff > 0 else 0

def pseudo_overlap_alpha(x, y):
    return x*y

def pseudo_overlap_pi(x, y):
    return x*(y**2)

def pseudo_overlap_phi(x, y):
    return (x**2)*y

def pseudo_overlap_psi(x, y):
    return np.min((x**2,y**2))

def pseudo_overlap_nu(x, y):
    return np.min((x**3,y**3))

def pseudo_overlap_epsilon(x, y):
    return 0.1*x*(y**2)+0.9*np.max((0,x+y-1))

def pseudo_overlap_delta(x, y):
    return np.max((np.min((x,y/2)),x+y-1))

def pseudo_overlap_gamma(x, y):
    denom = 0.5 * x + 1.5 * y
    if denom != 0:
        return (2 * x * y) / denom
    else:
        return 0

def pseudo_overlap_kappa(x, y):
    denom = x + (2* y)
    if denom != 0:
        return (3 * x * y) / denom
    else:
        return 0

def pseudo_overlap_mu(x, y):
    return ((x**2)*y)+((x**2)*y*(1-x)*(1-y))

def pseudo_overlap_my1(x, y):
    return np.max((x*(y**2),(x**2)*y))

def pseudo_overlap_my2(x, y):
    return np.min((x*(y**2),(x**2)*y))

def pseudo_overlap_beta(x, y):
    denom = x + y
    if denom != 0:
        return (2 * x * y) / denom
    else:
        return 0

def pseudo_overlap_my_min(x, y):
    return np.min((x,y))

def pseudo_overlap_my_third(x, y):
    return (x**3)*(y)

In [ ]:
def find_subset(f,y):
  return [i for i, value in enumerate(f) if value >= y]


def pop_choquet_integral(f, singleton_measures, pop_function_1,pop_function_2):
  y=DR_function(f)
  n = len(y)
  integral = 0.0
  previous_value = 0.0

  for i in range(n):
    val=pop_function_1(y[i],sugeno_measure(find_subset(f,y[i]),singleton_measures))-pop_function_2(previous_value,sugeno_measure(find_subset(f,y[i]),singleton_measures))
    previous_value=y[i]
    integral+=val

  return integral

In [ ]:
acc_values=list(accuracy_vectors.values())
print(acc_values)
print(matrix_M[0,0])


N = np.zeros((matrix_M.shape[0], matrix_M.shape[1]))

for i in range(matrix_M.shape[0]):
    for j in range(matrix_M.shape[1]):
        prvok=pop_choquet_integral(matrix_M[i,j],acc_values,pseudo_overlap_gamma,pseudo_overlap_delta)
        N[i,j]=prvok

print(N)

[0.4069007758268681, 0.43446304614128217, 0.500204164965292, 0.4679461004491629]
[0.e+00 7.e-05 0.e+00 0.e+00]
[[9.33283210e-05 7.66635592e-05 6.90405948e-01 ... 4.23327257e-01
  1.86646619e-04 0.00000000e+00]
 [1.01716568e-02 2.05516064e-01 1.04846158e+00 ... 2.74941034e-01
  3.19197909e-02 0.00000000e+00]
 [4.83209672e-04 2.95626620e-02 1.23590137e+00 ... 1.47235434e-01
  1.99444954e-02 1.19992944e-04]
 ...
 [9.33286728e-05 1.77668126e-02 7.58380694e-01 ... 3.02841259e-01
  1.19177073e-02 0.00000000e+00]
 [4.33323516e-05 1.69984628e-04 3.44702186e-03 ... 1.68753849e+00
  2.74916098e-01 2.05858257e-01]
 [1.66652975e-04 1.16255975e-03 1.30099321e-01 ... 1.03772910e+00
  2.77875224e-01 3.30011885e-02]]


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

predicted_indices = np.argmax(N, axis=1)
print("Predpovedané triedy (indexy stĺpcov):")
print(predicted_indices)

accuracy = accuracy_score(y_encoded, predicted_indices)
f1 = f1_score(y_encoded, predicted_indices, average='weighted')
conf_matrix = confusion_matrix(y_encoded, predicted_indices)
report = classification_report(y_encoded, predicted_indices)

print("\nAgregovaná ensemble klasifikácia - metriky:")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 score (weighted): {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(report)


Predpovedané triedy (indexy stĺpcov):
[3 2 2 ... 3 4 3]

Agregovaná ensemble klasifikácia - metriky:
Accuracy: 0.4935
F1 score (weighted): 0.4850
Confusion Matrix:
[[   2    3    8    6    1    0    0]
 [   2   26   91   41    3    0    0]
 [   3   40  814  520   76    4    0]
 [   1    8  580 1183  404   22    0]
 [   0    0   49  423  386   22    0]
 [   2    0    5   73   89    6    0]
 [   0    0    0    1    2    2    0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.20      0.10      0.13        20
           1       0.34      0.16      0.22       163
           2       0.53      0.56      0.54      1457
           3       0.53      0.54      0.53      2198
           4       0.40      0.44      0.42       880
           5       0.11      0.03      0.05       175
           6       0.00      0.00      0.00         5

    accuracy                           0.49      4898
   macro avg       0.30      0.26      0.27      4898
weig

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
